In [1]:
# dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
import numpy as np
from sklearn.svm import SVC

# set max columns
pd.set_option('display.max_columns', None)

# Import lyrics data
platinum_features_df = pd.read_csv('https://platinum-lyric-bucket.s3.us-east-2.amazonaws.com/platinum_features.csv')

In [2]:
platinum_features_df.head()

,Unnamed: 0,track_id,artist_name,song_title,song_year,feature_genre,feature_popularity,feature_duration,feature_key,feature_acousticness,feature_instrumentalness,feature_tempo,feature_mode,feature_danceability,feature_energy,feature_liveness,feature_loudness,feature_speechiness,feature_valence,feature_explicit,target_success,target_weeks,target_peak
0,0,TRRBOBU128F4293068,texas,i don't want a lover,1989,country,61,300600,7,0.1960,0.000487,120.484,1,0.756,0.470,0.1260,-12.615,0.0394,0.430,0,1,77,6
1,1,TRVCPQS128F4285928,the youngbloods,ride the wind,1988,rock,20,396600,2,0.9100,0.651000,119.033,1,0.558,0.307,0.0866,-20.492,0.0343,0.674,0,0,0,0
2,2,TRZRMWW128F426E797,babyface,tender lover,1990,pop,37,259267,5,0.2260,0.000422,102.459,1,0.743,0.860,0.0513,-6.346,0.0445,0.687,0,1,17,89
3,3,TRVSRVI128F4261843,reo speedwagon,one lonely night,1984,country,31,201467,5,0.0561,0.000149,76.051,1,0.408,0.579,0.0712,-10.277,0.0260,0.397,0,1,19,16
4,4,TRSHXOI128F146B1AE,john waite,change,1982,rock,49,196693,11,0.2460,0.000000,149.028,1,0.403,0.848,0.6160,-11.615,0.0625,0.622,0,1,54,10


In [3]:
# drop non-word columns
features_df = platinum_features_df.drop(['feature_popularity', 'Unnamed: 0', 'track_id', 'artist_name', 'song_title', 'song_year', 'target_weeks', 'target_peak'], 1)

# Binary encoding using Pandas (single column)
features_df = pd.get_dummies(features_df, columns=["feature_genre"])
features_df

,feature_duration,feature_key,feature_acousticness,feature_instrumentalness,feature_tempo,feature_mode,feature_danceability,feature_energy,feature_liveness,feature_loudness,feature_speechiness,feature_valence,feature_explicit,target_success,feature_genre_blues,feature_genre_country,feature_genre_edm,feature_genre_electronic,feature_genre_folk,feature_genre_hip hop,feature_genre_jam,feature_genre_jazz,feature_genre_latin,feature_genre_metal,feature_genre_pop,feature_genre_punk,feature_genre_rap,feature_genre_reggae,feature_genre_rhythm and blues,feature_genre_rock
0,300600,7,0.19600,0.000487,120.484,1,0.756,0.470,0.1260,-12.615,0.0394,0.430,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,396600,2,0.91000,0.651000,119.033,1,0.558,0.307,0.0866,-20.492,0.0343,0.674,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,259267,5,0.22600,0.000422,102.459,1,0.743,0.860,0.0513,-6.346,0.0445,0.687,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,201467,5,0.05610,0.000149,76.051,1,0.408,0.579,0.0712,-10.277,0.0260,0.397,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,196693,11,0.24600,0.000000,149.028,1,0.403,0.848,0.6160,-11.615,0.0625,0.622,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687,199067,9,0.00962,0.000000,140.219,1,0.395,0.955,0.3600,-3.799,0.0488,0.658,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8688,288080,4,0.02280,0.224000,113.250,0,0.704,0.709,0.0832,-10.453,0.0589,0.848,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8689,318773,11,0.31500,0.000006,177.872,1,0.391,0.514,0.0567,-6.398,0.0447,0.437,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8690,202760,7,0.12300,0.000031,185.034,1,0.443,0.885,0.2800,-6.172,0.0414,0.536,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# Seperate features and target
y = features_df["target_success"]
X = features_df.drop(columns="target_success")

# Split data into testing and training
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)
X_train.shape

(6519, 29)

In [5]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Random forest model

In [6]:
# Creating the decision tree classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 0, ..., 1, 1, 1])

In [7]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
accuracy_score(y_test, predictions)

0.6907501150483203

In [8]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_

# Sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09392521343331216, 'feature_duration'),
 (0.09235527882533268, 'feature_danceability'),
 (0.08953662835838501, 'feature_acousticness'),
 (0.0893009669759923, 'feature_speechiness'),
 (0.08719692946330021, 'feature_energy'),
 (0.0830058329403259, 'feature_instrumentalness'),
 (0.08152161747988391, 'feature_valence'),
 (0.0797540730427805, 'feature_loudness'),
 (0.07812576923779825, 'feature_liveness'),
 (0.07780376721539606, 'feature_tempo'),
 (0.04629553008605502, 'feature_key'),
 (0.02684385150305692, 'feature_genre_pop'),
 (0.011473464590204966, 'feature_mode'),
 (0.009107536240150635, 'feature_genre_rock'),
 (0.00870601724103195, 'feature_genre_metal'),
 (0.008407860328712576, 'feature_genre_country'),
 (0.0054140394897028874, 'feature_explicit'),
 (0.004548498373635888, 'feature_genre_latin'),
 (0.004412828664016654, 'feature_genre_folk'),
 (0.0043432725409339335, 'feature_genre_blues'),
 (0.0035536411588197227, 'feature_genre_rhythm and blues'),
 (0.0033575078920412213, 'featu

# SVC Model

In [9]:
# Instantiate a linear SVM model
model = SVC(kernel='linear')

# Fit the data
model.fit(X_train_scaled, y_train)

# Make predictions using the test data
y_pred = model.predict(X_test_scaled)
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,1,1
1,1,0
2,0,1
3,0,0
4,1,1


In [10]:
accuracy_score(y_test, y_pred)

0.6576161988034974

# NN Model

In [11]:
# # Creating StandardScaler instance
# scaler = MinMaxScaler()

# # Fitting Standard Scaller
# X_scaler = scaler.fit(X_train)

# # Scaling data
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features
hidden_nodes_layer2 = number_input_features

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, np.asarray(y_train), epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,np.asarray(y_test),verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 6519 samples
Epoch 1/100
6519/6519 [==============================] - 2s 254us/sample - loss: 0.6690 - accuracy: 0.5907
Epoch 2/100
6519/6519 [==============================] - 1s 97us/sample - loss: 0.6401 - accuracy: 0.6377
Epoch 3/100
6519/6519 [==============================] - 1s 100us/sample - loss: 0.6292 - accuracy: 0.6529
Epoch 4/100
6519/6519 [==============================] - 1s 95us/sample - loss: 0.6228 - accuracy: 0.6584
Epoch 5/100
6519/6519 [==============================] - 1s 93us/sample - loss: 0.6203 - accuracy: 0.6618
Epoch 6/100
6519/6519 [==============================] - 1s 96us/sample - loss: 0.6180 - accuracy: 0.6579
Epoch 7/100
6519/6519 [==============================] - 1s 102us/sample - loss: 0.6133 - accuracy: 0.6636
Epoch 8/100
6519/6519 [==============================] - 1s 101us/sample - loss: 0.6116 - accuracy: 0.6659
Epoch 9/100
6519/6519 [==============================] - 1s 100us/sample - loss: 0.6101 - accuracy: 0.6677
Epoch 10/100
6519/6

6519/6519 [==============================] - 1s 94us/sample - loss: 0.5618 - accuracy: 0.7000
Epoch 77/100
6519/6519 [==============================] - 1s 89us/sample - loss: 0.5609 - accuracy: 0.7050
Epoch 78/100
6519/6519 [==============================] - 1s 99us/sample - loss: 0.5601 - accuracy: 0.7052
Epoch 79/100
6519/6519 [==============================] - 1s 97us/sample - loss: 0.5603 - accuracy: 0.7056
Epoch 80/100
6519/6519 [==============================] - 1s 89us/sample - loss: 0.5610 - accuracy: 0.7042
Epoch 81/100
6519/6519 [==============================] - 1s 110us/sample - loss: 0.5583 - accuracy: 0.7090
Epoch 82/100
6519/6519 [==============================] - 1s 126us/sample - loss: 0.5594 - accuracy: 0.7036
Epoch 83/100
6519/6519 [==============================] - 1s 96us/sample - loss: 0.5569 - accuracy: 0.7036
Epoch 84/100
6519/6519 [==============================] - 1s 108us/sample - loss: 0.5577 - accuracy: 0.7093
Epoch 85/100
6519/6519 [=======================